In [1]:
import sys
import os
sys.path.append(os.path.abspath('..'))

In [50]:
import torch
import src.load as load
import src.utils as utils


In [3]:
torch.backends.mps.is_available()

True

In [4]:
device = torch.device('mps')

In [5]:
train_year, test_year, _, _, _ = load.separate_year('../data/processed/', device=device)

In [6]:
train_prop, test_prop, _, _, _ = load.separate_prop('../data/processed/', device=device)


In [31]:
num_epochs = 500


In [51]:
report_prop, kilo_state_prop, schedule_models_prop, final_models_prop = utils.test_model(train_prop, test_prop, num_epochs, device,'sandbox/overnight_training_prop')

Epoch 0 completed
Epoch 1 completed
Kilo triggered at epoch -1 and best kilo model saved at epoch 1. Overall, it triggered 0 times
Schedule triggered at epoch -1 and best schedule model saved at epoch 1. Overall, it triggered 0 times
Best final model saved at epoch 1. Overall, it triggered 0 times


In [ ]:
report_year, kilo_state_year, schedule_models_year, final_models_year = utils.test_model(train_year, test_year, num_epochs, device,'sandbox/overnight_training_year')